In [1]:
from sim_functions import *
from helpers import *
from Variational_Functions import *

# Simulation for Response to Reviewer 1 Major Comment 1, Review Figures 1, 2, 3

In [2]:
nm = 125
def dict_series(dict, n, method, folds, lp):
    dict = {key: float(item) for key, item in dict.items()}
    dict["n"] = n
    dict["method"] = method
    dict["fold"] = folds
    dict["lp"] = lp
    return pd.Series(dict)
    

ddp_list =  [12, 36, 48]
D = 30
n_list = [300, 400, 500]
#df = pd.DataFrame(columns=['rmse', 'nll','crps', "cov", "unc", "loss", "n", "method", "fold", "lp"])
for n in n_list:
    for Lp in ddp_list:
        df = pd.DataFrame(columns=['rmse', 'nll','crps', "cov", "unc", "loss", "n", "method", "fold", "lp"])
        for f in range(10):
            X, Y, Xtest, Ytest = draw_exp(ntrain = n, ntest = 300, D = D, stride = int(30//3))
                    
            try:
                vim3_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = 4, J = 3,nm = nm, ft = 1.75, iters = 5000, verbose = False, lr = .005, retry = True, lam_init = .01)
            except:
                vim3_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = int(Lp/3), J = 3, nm = nm, ft = 1.75, iters =4500, 
                                        lr = .0025, retry = False)
                
            try:
                vim4_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 =  4, J = 4, nm = nm, ft = 1.75, iters = 5000, verbose = False, lr = .005, retry = True, lam_init = .01)
            except:
                vim4_dict = vim_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                        d1 = int(Lp/4), J = 4, nm = nm, ft = 1.75, iters =4500, 
                                        lr = .0025, retry = False, lam_init = .2)
            dict_list = [vim3_dict, vim4_dict]
            print(dict_list)
            dict_dict = [dict_series(dict_list[i-3], n = n, method = "J={}".format(i), folds = f, lp = Lp) for i in range(3, 5)]
            
            result = pd.concat(dict_dict, ignore_index = True, axis = 1).T
            df = pd.concat([df, result], ignore_index=True)
            df.to_csv("reviewer_results/review_lp_{}_{}_25.csv".format(Lp, n, 25))
            print(df)

  0%|          | 0/5000 [00:00<?, ?it/s]

first optimization


  0%|          | 0/5000 [00:01<?, ?it/s]


RuntimeError: CUDA error: an illegal memory access was encountered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


# Simulation for Reviewer 1 Minor Comment 3, Response Figure 4

In [ ]:
df = pd.DataFrame(columns=['train_rmse', "cov", "method"])

for i in range(10):
    X, Y, Xtest, Ytest = draw_exp(ntrain = 300, ntest = 300, D = 25, stride = int(25//3))
    
    
    try:
        dle_dict = dle_stats(X = X, Y = Y, Xtest = Xtrain, Ytest=  Ytrain, 
                d1 = 8, retry = True, train_stats = True)
    except:
        dle_dict = dle_stats(X = X, Y = Y, Xtest = Xtest, Ytest=  Ytest, 
                                d1 = 8, ft = 1.25, retry = False, train_stats = True)
    dle = pd.Series({"train_rmse": dle_dict["train_rmse"], "cov": dle_dict["cov"], "method": "DR-GP"})
    
    try:
        dpa_dict = dpa_stats(X = X, Y = Y, Xtest = Xtrain, Ytest = Ytrain.float(), 
                             num_proj = 8, retry = True, train_stats=  True)
    except:
        dpa_dict = dpa_stats(X = X, Y = Y, Xtest = Xtest, Ytest = Ytest.float(),
                             num_proj = 8, train_stats = True)
    dpa = pd.Series({"train_rmse": dpa_dict["train_rmse"], "cov": dpa_dict["cov"], "method": "DPA-GP"})

    df = pd.concat([df, pd.DataFrame(dle).T, pd.DataFrame(dpa).T], ignore_index=True)
df.to_csv("reviewer_results/numerical_instability.csv")